In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [7]:
train_lre_30sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [8]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [9]:
y_30sec_labels = train_lre_30sec["lang"]

In [10]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [11]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [12]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [13]:
y_30sec_labels=le.transform(y_30sec_labels)

In [14]:
y_30sec_labels.shape

(114276,)

In [15]:
nb_classes = 14
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [16]:
Y_train_labels.shape

(114276, 14)

In [17]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [18]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [19]:
#Shuffle the Dataset
X_train_10sec_F,  X_train_30sec_F, Y_train_labels = shuffle(X_train_10sec, X_train_30sec, Y_train_labels, random_state=0)

In [20]:
import torch.nn as nn

In [142]:
class FC_Network(torch.nn.Module):

    def __init__(self,):
        super(FC_Network, self).__init__()

        D_in =  514
        D_out =500      
        num_nodes_fnn = 512
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_4 = torch.nn.Linear(num_nodes_fnn,num_nodes_fnn)
        self.bn_4 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()

    def forward(self, x, labels):
        out = torch.cat([x, labels], 1)
        #out = x.view(x.size(0), -1)
        out = self.bn_1(self.relu(self.layer_1(out)))
        out = self.bn_2(self.relu(self.layer_2(out)))
        out = self.bn_3(self.relu(self.layer_3(out)))
        out = self.bn_4(self.relu(self.layer_4(out)))
        out = self.tanh(self.output_layer(out))

        #out = (self.relu(self.layer_1(out)))
        #out = (self.relu(self.layer_2(out)))
        #out = (self.relu(self.layer_3(out)))
        #out = (self.relu(self.layer_4(out)))
        #out = (self.output_layer(out))
        #out = out.view(x.size())
        return out

In [143]:
S_encoder = FC_Network()

In [144]:
S_encoder.cuda()

FC_Network(
  (layer_1): Linear(in_features=514, out_features=4096)
  (bn_1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=4096, out_features=4096)
  (bn_2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=4096, out_features=4096)
  (bn_3): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=4096, out_features=4096)
  (bn_4): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=4096, out_features=500)
  (relu): ReLU()
  (tanh): Tanh()
)

In [145]:
learning_rate = 0.00005
betas = (0.5, 0.999)

In [146]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
S_solver = torch.optim.Adam(S_encoder.parameters(), lr=learning_rate, betas=betas)
#T_solver = torch.optim.Adam(T_encoder.parameters(), lr=learning_rate/2, betas=betas)

In [147]:
len(X_train_10sec)

114276

In [148]:
len(X_train_30sec)

114276

In [149]:
mb_size = 256
batch_size = mb_size

In [150]:
#Implementation of Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(514, 512)
        self.fc2 = torch.nn.Linear(512,512)
        self.fc3 = torch.nn.Linear(512,1)
        
    def forward(self, x ,labels):
        inputs = torch.cat([x, labels], 1)
        return F.sigmoid(self.fc3(F.relu(self.fc2(F.relu(self.fc1(inputs))))))


In [151]:
D = Discriminator()

In [152]:
D.cuda()

Discriminator(
  (fc1): Linear(in_features=514, out_features=512)
  (fc2): Linear(in_features=512, out_features=512)
  (fc3): Linear(in_features=512, out_features=1)
)

In [153]:
S_encoder

FC_Network(
  (layer_1): Linear(in_features=514, out_features=4096)
  (bn_1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=4096, out_features=4096)
  (bn_2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=4096, out_features=4096)
  (bn_3): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=4096, out_features=4096)
  (bn_4): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=4096, out_features=500)
  (relu): ReLU()
  (tanh): Tanh()
)

In [154]:
ones_label = Variable(torch.ones(mb_size, 1))
zeros_label = Variable(torch.zeros(mb_size,1))
#ones_label_fake = Variable(torch.ones(mb_size*2))

In [155]:
ones_label = ones_label.cuda()
zeros_label = zeros_label.cuda()

In [156]:
# Optimizers
#T_solver = torch.optim.Adam(T_encoder.parameters(), lr=learning_rate, betas=betas)
D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [157]:
from torch.nn.functional import binary_cross_entropy

In [158]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train_10sec_F) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train_10sec_F[start:end]
        
        labels = Y_train_labels[start:end]
        
        c = X_train_30sec_F[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))
        labels = Variable(torch.from_numpy(labels.astype('float32')))
        X = X.cuda()
        c = c.cuda()
        labels = labels.cuda()
        # Dicriminator forward-loss-backward-update
        S_sample = S_encoder(X, labels)
        #T_sample = T_encoder(c)
        D_real = D(c, labels)
        D_fake = D(S_sample, labels)

        D_loss_real = binary_cross_entropy(D_real, ones_label)
        D_loss_fake = binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()
        D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        S_sample = S_encoder(X, labels)
        D_fake = D(S_sample, labels)
        #D_fake = D(G_sample, c)
        #T_loss = criterion(T_sample, c)
        S_loss = binary_cross_entropy(D_fake, ones_label)

        S_loss.backward()
        S_solver.step()
        
        S_encoder.zero_grad()
        # Housekeeping - reset gradient
        D.zero_grad()

        

        it+= batch_size
    print('S_loss: {}'.format(S_loss.cpu().data.numpy()),'D_loss: {}'.format(D_loss.cpu().data.numpy()))
           

epoch: 0
S_loss: [ 0.70011902] D_loss: [ 0.85357523]
epoch: 1
S_loss: [ 0.71106029] D_loss: [ 0.93594038]
epoch: 2
S_loss: [ 0.71963704] D_loss: [ 1.01905131]
epoch: 3
S_loss: [ 0.81437236] D_loss: [ 1.03888726]
epoch: 4
S_loss: [ 0.94948852] D_loss: [ 1.01799083]
epoch: 5
S_loss: [ 1.12310195] D_loss: [ 0.94241285]
epoch: 6
S_loss: [ 1.22783816] D_loss: [ 0.91288382]
epoch: 7
S_loss: [ 1.39030278] D_loss: [ 0.89169592]
epoch: 8
S_loss: [ 1.21421719] D_loss: [ 1.10677004]
epoch: 9
S_loss: [ 1.00437713] D_loss: [ 1.27748597]
epoch: 10
S_loss: [ 0.98950469] D_loss: [ 1.19952953]
epoch: 11
S_loss: [ 0.91357547] D_loss: [ 1.26835191]
epoch: 12
S_loss: [ 0.91865015] D_loss: [ 1.19774556]
epoch: 13
S_loss: [ 0.99326849] D_loss: [ 1.14076161]
epoch: 14
S_loss: [ 1.06863272] D_loss: [ 1.03823233]
epoch: 15
S_loss: [ 1.19465995] D_loss: [ 0.95088506]
epoch: 16
S_loss: [ 1.36266732] D_loss: [ 0.78799355]
epoch: 17
S_loss: [ 1.48689938] D_loss: [ 0.70869565]
epoch: 18
S_loss: [ 1.69660306] D_loss

In [ ]:
S_encoder

In [ ]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec_F))

In [ ]:
S_encoder.eval()

In [ ]:
gen_samples = S_encoder(X_train_30sec_gen.cuda())

In [ ]:
gen_samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X_gen = gen_samples.cpu().data.numpy()

In [ ]:
type(train_X_gen)

In [ ]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [ ]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_05Jan_10sec.csv')


In [ ]:
val_lre.head()

In [ ]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [ ]:
X_val = X_val.values
#X_eval = X_eval.values

In [ ]:
#X_eval.shape

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

In [ ]:
le.classes_

In [ ]:
y_val_labels = le.transform(y_val)

In [ ]:
y_val_labels[0:25]

In [ ]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
#Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [ ]:
Y_val

In [ ]:
X_val = X_val.astype('float32')


In [ ]:
X_val_10sec_gen = Variable(torch.from_numpy(X_val))

In [ ]:
gen_samples_val = S_encoder(X_val_10sec_gen.cuda())

In [ ]:
gen_samples_val.data.shape

In [ ]:
X_val_gen = gen_samples_val.cpu().data.numpy()

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch = 15

In [ ]:
#train_X_gen
#Checking Accuracy with augmented data 
history = model.fit(train_X_gen, Y_train_labels , batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val_gen , Y_val),callbacks=[saveBestModel])

In [ ]:
score = model.evaluate(X_val_gen, Y_val, verbose=0)
print('Acc %:', score[1])

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


In [ ]:
classif = SVC(C=1, kernel='rbf', gamma='auto', coef0=1, shrinking=True, random_state=0,
                      probability=False, tol=1e-3, cache_size=1e4, class_weight='balanced')
classif.fit(X_train_10sec_F, y_30sec_labels)

In [ ]:
classif.classes_

In [ ]:
acc = metrics.accuracy_score(y_val_labels, classif.predict(X_val))

In [ ]:
acc